004に対して以下を追加

承知いたしました。グローバルアイテム埋め込みの更新をFedAvg（Federated Averaging）に基づいて行う実装を示します。

FedAvgでは、各クライアントからアップロードされたモデルパラメータ（ここでは`local_item_embedding.weight`）を、そのクライアントのデータセットサイズ（または他の貢献度指標）で重み付けして平均します。

**変更点:**

1.  **`ClientModel.forward` の変更なし**: クライアントモデルからアップロードされる重みはこれまで通り`self.local_item_embedding.weight`です。
2.  **`Server.aggregate_item_embeddings` の修正**:
      * `user_graph_adj` と `sorted_user_ids` を使ったグラフ畳み込みのロジックは残します。
      * そのグラフ畳み込みの結果得られた `R_tensor` を直接平均するのではなく、FedAvg の重み付け平均を適用します。
      * 重み付けには、各クライアント（ユーザー）が持つインタラクションの総数を使用します。これは、`len(dataloader.dataset)` で取得できます。
      * ※ **ただし、論文はFedAVG的なやり方は実行しておらず、004のように単純に平均をとって$\theta_{global}$を更新している$**
3.  **学習ループの修正**:
      * 各クライアントが自身のデータセットサイズをサーバーに報告できるようにします。
      * サーバーは、報告されたデータセットサイズを使って重み付き平均を計算します。

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, Subset
from transformers import AutoModel, AutoTokenizer
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict

# 軽量 LLM 埋め込みモデルのロード (変更なし)
plm_model_name = "sentence-transformers/all-MiniLM-L6-v2"
plm_tokenizer = AutoTokenizer.from_pretrained(plm_model_name)
plm_model = AutoModel.from_pretrained(plm_model_name)

# PLMは学習済みモデルのため、勾配計算を無効化
for param in plm_model.parameters():
    param.requires_grad = False

plm_embedding_dim = plm_model.config.hidden_size
print(f"PLM embedding dimension: {plm_embedding_dim}")


class ClientModel(nn.Module):
    def __init__(self, num_items, item_embedding_dim, plm_model, plm_embedding_dim, joint_embedding_output_dim):
        super(ClientModel, self).__init__()
        self.plm_model = plm_model
        
        # Joint Embedding Layer (module parameter θ_user)
        self.user_joint_embedding_linear = nn.Linear(plm_embedding_dim, joint_embedding_output_dim)
        
        # Item Embedding Layer (module parameter θ_item)
        self.local_item_embedding = nn.Embedding(num_items, item_embedding_dim)

        # User Feature Refinement MLP (module parameter θ_umlp)
        self.user_mlp = nn.Sequential(
            nn.Linear(joint_embedding_output_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 32)
        )

        # Predictive Scoring Function (module parameter θ_score)
        self.prediction_mlp = nn.Sequential(
            nn.Linear(item_embedding_dim + joint_embedding_output_dim, 16),
            nn.ReLU(),
            nn.Linear(16, 8),
            nn.ReLU(),
            nn.Linear(8, 1)
        )

    def forward(self, user_ids, item_ids, user_texts_batch): 
        encoded_input = plm_tokenizer(user_texts_batch, padding=True, truncation=True, return_tensors='pt')
        plm_output = self.plm_model(**encoded_input).last_hidden_state[:, 0, :]

        user_raw_embedding = self.user_joint_embedding_linear(plm_output)
        user_embedding = self.user_mlp(user_raw_embedding)

        item_embedding = self.local_item_embedding(item_ids)

        combined_features = torch.cat((user_embedding, item_embedding), dim=1)
        logits = self.prediction_mlp(combined_features)
        predictions = torch.sigmoid(logits)

        return predictions, self.user_joint_embedding_linear.weight, self.local_item_embedding.weight


class Server:
    def __init__(self, num_users, num_items, item_embedding_dim, joint_embedding_output_dim):
        self.global_item_embedding = nn.Embedding(num_items, item_embedding_dim)
        self.num_users = num_users
        self.num_items = num_items
        self.item_embedding_dim = item_embedding_dim
        self.joint_embedding_output_dim = joint_embedding_output_dim

    def build_user_relationship_graph(self, user_linear_weights_map):
        """
        各ユーザーのuser_joint_embedding_linear.weightからユーザー関係グラフを構築します。
        論文の式 (15) に基づいています。 [cite: 106]

        Args:
            user_linear_weights_map (dict): {user_id: user_joint_embedding_linear.weight.data (d1, d2) Tensor}

        Returns:
            np.ndarray: ユーザーグラフの隣接行列 (NumPy配列)
            list: グラフのノード順に対応するユーザーIDのリスト
        """
        sorted_user_ids = sorted(user_linear_weights_map.keys())
        if not sorted_user_ids:
            return np.zeros((0, 0)), []

        user_weight_vectors = np.array([
            user_linear_weights_map[u_id].flatten().cpu().numpy() for u_id in sorted_user_ids
        ])

        similarity_matrix = cosine_similarity(user_weight_vectors)
        user_graph_adj = similarity_matrix

        return user_graph_adj, sorted_user_ids

    def aggregate_item_embeddings(self, user_local_item_weights, user_graph_adj, sorted_user_ids, client_data_sizes):
        """
        ユーザー関係グラフに基づいてアイテム埋め込みを集約し、FedAvgでグローバル埋め込みを更新します。

        Args:
            user_local_item_weights (dict): {user_id: local_item_embedding.weight.data (Tensor)}
            user_graph_adj (np.ndarray): ユーザーグラフの隣接行列
            sorted_user_ids (list): user_graph_adj のノード順に対応するユーザーIDのリスト
            client_data_sizes (dict): {user_id: そのユーザーのデータセットサイズ}

        Returns:
            torch.Tensor: 更新されたグローバルアイテム埋め込みの重み
        """
        if not user_local_item_weights:
            return self.global_item_embedding.weight.data

        # グラフの順序に合わせて各ユーザーのアイテム埋め込みを行列Aとしてまとめる
        item_embedding_matrix_A = torch.stack([
            user_local_item_weights[u_id] for u_id in sorted_user_ids
        ]) # (num_users, num_items, item_embedding_dim)

        # グラフの正規化 (S'')今回は簡易的に行なっている
        # INFO:論文でのグラフの正規化の流れ
        # 1. user_graph_adjのi番目の横ベクトルを抽出
        # 2. 横ベクトルの値を降順に並べた時のtop-kとなるインデックスのListを取得
        # 3. top-kのインデックスに対応するところの値を 1/neighborhood_sizeと固定値とし、他のインデックスを0にした横ベクトルを作成
        # 4. それを全ユーザに対して実行し、縦積みしていく.
        row_sums_graph = np.sum(user_graph_adj, axis=1, keepdims=True)
        row_sums_graph[row_sums_graph == 0] = 1 
        normalized_user_graph_adj = user_graph_adj / row_sums_graph

        normalized_user_graph_adj_tensor = torch.tensor(normalized_user_graph_adj, dtype=torch.float32)

        # グラフ畳み込み (R = S'' A)
        # INFO:
        # - S'': ユーザ数 x ユーザ数の正規化された隣接行列
        # - A: ユーザ数 x アイテム数 x アイテム埋め込み次元
        R_tensor = torch.einsum('ij, jkd -> ikd', normalized_user_graph_adj_tensor, item_embedding_matrix_A)

        # グローバルアイテム埋め込みの更新 (FedAvg: D = degree matrix at the time of aggregation)
        # R_tensorの各行は、各ユーザーがグラフ畳み込みによって集約されたアイテム埋め込み
        # FedAvgでは、これらをクライアントのデータセットサイズで重み付けして平均する

        total_data_size = sum(client_data_sizes[u_id] for u_id in sorted_user_ids)
        if total_data_size == 0:
            return self.global_item_embedding.weight.data # データがない場合は更新しない

        weighted_sum_item_embeddings = torch.zeros_like(self.global_item_embedding.weight.data)

        for i, u_id in enumerate(sorted_user_ids):
            weight = client_data_sizes[u_id] / total_data_size
            weighted_sum_item_embeddings += weight * R_tensor[i] # R_tensorのi番目の行はユーザーu_idの集約されたアイテム埋め込み

        new_global_item_embedding_weight = weighted_sum_item_embeddings

        # サーバーのグローバルアイテム埋め込みを直接更新
        self.global_item_embedding.weight.data.copy_(new_global_item_embedding_weight)

        return self.global_item_embedding.weight.data


# データセットの準備とクライアントへの分割 (1クライアント1ユーザー)
num_users = 100
num_items = 50
num_clients = num_users 

user_texts = {i: f"This user likes movies about {i % 5} and enjoys {i % 3}." for i in range(num_users)}

interactions_list = []
user_interaction_history = defaultdict(list) 

for u_id in range(num_users):
    for i_id in range(num_items):
        if np.random.rand() > 0.7:
            interactions_list.append([u_id, i_id, 1])
            user_interaction_history[u_id].append(i_id) 
        else:
            interactions_list.append([u_id, i_id, 0])

interactions = torch.tensor(interactions_list, dtype=torch.float32)

client_user_map = {} 
client_datasets = {}
client_original_data_sizes = {} # 各クライアントのデータセットサイズを保存
for u_id in range(num_users):
    client_id = u_id 
    client_user_map[client_id] = u_id 
    
    client_interactions_indices = [i for i, (u, _, _) in enumerate(interactions_list) if u == u_id]
    
    if not client_interactions_indices:
        print(f"Warning: User {u_id} has no interactions. Client {client_id} will have an empty dataset.")
        client_subset = TensorDataset(
            torch.empty(0, dtype=torch.long), 
            torch.empty(0, dtype=torch.long), 
            torch.empty(0, dtype=torch.float32)
        )
        client_original_data_sizes[client_id] = 0
    else:
        user_interaction_data_for_client = []
        for idx in client_interactions_indices:
            u_id_data, i_id_data, label_data = interactions_list[idx]
            user_interaction_data_for_client.append((u_id_data, i_id_data, label_data))

        users_tensor = torch.tensor([d[0] for d in user_interaction_data_for_client], dtype=torch.long)
        items_tensor = torch.tensor([d[1] for d in user_interaction_data_for_client], dtype=torch.long)
        labels_tensor = torch.tensor([d[2] for d in user_interaction_data_for_client], dtype=torch.float32)

        client_subset = TensorDataset(users_tensor, items_tensor, labels_tensor)
        client_original_data_sizes[client_id] = len(client_subset)
    
    client_datasets[client_id] = DataLoader(client_subset, batch_size=min(32, max(1, len(client_subset))), shuffle=True)

print(f"Number of users: {num_users}")
print(f"Number of items: {num_items}")
print(f"Total interactions: {len(interactions)}")
print(f"Number of clients (1 client per user): {num_clients}")


# モデルのハイパーパラメータ
item_embedding_dim = 32
joint_embedding_output_dim = 32 

# サーバーのインスタンス化
server = Server(num_users, num_items, item_embedding_dim, joint_embedding_output_dim)

# 各クライアントのモデルを辞書で保持
client_models = {}
client_optimizers = {}
for client_id in range(num_clients):
    client_models[client_id] = ClientModel(
        num_items,
        item_embedding_dim,
        plm_model,
        plm_embedding_dim,
        joint_embedding_output_dim
    )
    # NOTE:
    # クライアントごとに最適化するパラメータを設定
    # ここでは、user_joint_embedding_linear, local_item_embedding, prediction_layer が対象
    # 単純にoptim.Adam(params = client_models[client_id].parameters(), lr=0.001)とすると、
    # PLMも学習可能パラメータとなってしまうので、
    # PLMのパラメータを除外したパラメータのみを取得してから、設定する.
    trainable_params = [
        p for name, p in client_models[client_id].named_parameters()
        if not name.startswith('plm_model.')
    ]

    client_optimizers[client_id] = optim.Adam(
        params=trainable_params,
        lr=0.001
    )

# 学習ループ (フェデレーテッド学習ラウンド)
num_communication_rounds = 10
local_epochs = 1 

for round_num in range(num_communication_rounds):
    print(f"\n--- Communication Round {round_num + 1}/{num_communication_rounds} ---")

    # サーバーからグローバルアイテム埋め込みをクライアントに配布
    for client_id in range(num_clients):
        client_models[client_id].local_item_embedding.weight.data.copy_(server.global_item_embedding.weight.data)

    user_linear_weights_for_graph = {} # {user_id: user_joint_embedding_linear.weight.data (d1, d2) Tensor}
    user_local_item_weights_to_server = {} # {user_id: local_item_embedding.weight.data (Tensor)}
    client_reported_data_sizes = {} # クライアントが報告するデータセットサイズ

    # クライアントのローカル学習
    for client_id in range(num_clients):
        model = client_models[client_id]
        optimizer = client_optimizers[client_id]
        dataloader = client_datasets[client_id]

        model.train()
        local_loss = 0

        current_user_id = client_user_map[client_id] 

        # クライアントが持つデータセットサイズを報告
        client_reported_data_sizes[current_user_id] = client_original_data_sizes[client_id]

        if len(dataloader.dataset) == 0:
            print(f"  Client {client_id} (User {current_user_id}) has no interactions, skipping local training.")
            # 訓練されなかったクライアントのために、現在のモデルの重み（グローバル初期化時と同じ）をアップロード
            user_linear_weights_for_graph[current_user_id] = model.user_joint_embedding_linear.weight.data.clone().flatten()
            user_local_item_weights_to_server[current_user_id] = model.local_item_embedding.weight.data.clone()
            continue

        for epoch in range(local_epochs):
            for user_ids_batch, item_ids_batch, labels_batch in dataloader:
                assert torch.all(user_ids_batch == current_user_id) 
                current_user_texts = [user_texts[uid.item()] for uid in user_ids_batch]

                optimizer.zero_grad()
                predictions, user_joint_embedding_linear_weight, local_item_embedding_weight = model(
                    user_ids_batch, item_ids_batch, current_user_texts
                )

                loss = nn.BCELoss()(predictions.squeeze(), labels_batch)

                lambda_reg = 0.01 

                regularization_term = torch.mean(
                    (local_item_embedding_weight - server.global_item_embedding.weight.data)**2
                )

                loss = loss + lambda_reg * regularization_term

                loss.backward()
                optimizer.step()
                local_loss += loss.item()

        user_linear_weights_for_graph[current_user_id] = user_joint_embedding_linear_weight.data.clone()
        user_local_item_weights_to_server[current_user_id] = local_item_embedding_weight.data.clone()

        print(f"  Client {client_id} (User {current_user_id}) local loss: {local_loss / len(dataloader):.4f}")

    # サーバーでの処理
    user_graph_adj, sorted_user_ids_for_graph = server.build_user_relationship_graph(
        user_linear_weights_for_graph
    )

    server.aggregate_item_embeddings(
        user_local_item_weights_to_server, 
        user_graph_adj, 
        sorted_user_ids_for_graph,
        client_reported_data_sizes # データセットサイズをサーバーに渡す
    )

    print(f"Round {round_num + 1} completed. Global item embeddings updated.")

print("Federated training completed.")

PLM embedding dimension: 384
Number of users: 100
Number of items: 50
Total interactions: 5000
Number of clients (1 client per user): 100

--- Communication Round 1/10 ---
  Client 0 (User 0) local loss: 0.6820
  Client 1 (User 1) local loss: 0.7625
  Client 2 (User 2) local loss: 0.7681
  Client 3 (User 3) local loss: 0.6766
  Client 4 (User 4) local loss: 0.7052
  Client 5 (User 5) local loss: 0.7019
  Client 6 (User 6) local loss: 0.7625
  Client 7 (User 7) local loss: 0.7510
  Client 8 (User 8) local loss: 0.6316
  Client 9 (User 9) local loss: 0.6711
  Client 10 (User 10) local loss: 0.6732
  Client 11 (User 11) local loss: 0.7028
  Client 12 (User 12) local loss: 0.6996
  Client 13 (User 13) local loss: 0.6873
  Client 14 (User 14) local loss: 0.6422
  Client 15 (User 15) local loss: 0.7532
  Client 16 (User 16) local loss: 0.8253
  Client 17 (User 17) local loss: 0.6690
  Client 18 (User 18) local loss: 0.7269
  Client 19 (User 19) local loss: 0.6631
  Client 20 (User 20) local 